In [1]:
import os
import joblib
import numpy as np
from model_settings import ms
from pathlib import Path
ms.find_root(Path())
models_dir = os.path.join(ms.root,ms.trained_models)
models = [f for f in os.listdir(models_dir) if f.find('.')==-1]
for i,m in enumerate(models):
    print(f"{i}  {m}")
i = int(input('select model: '))
model_dir = os.path.join(models_dir,models[i])
pickle = os.path.join(model_dir,models[i]+'.pkl')

def test_performance(model,net):
    inpred = net.predict(model['train_X'])
    intar = model['train_data'][model['target_name']].values
    indiff = inpred-intar
    inRMSE = np.sqrt(np.mean(indiff**2))
    inMAE = np.mean(np.abs(indiff))
    print(f"in sample\n   RMSE:\n      {inRMSE}")
    print(f"   MAE:\n      {inMAE}")

    outpred = net.predict(model['test_X'])
    outtar = model['test_data'][model['target_name']].values
    outdiff = outpred-outtar
    outRMSE = np.sqrt(np.mean(outdiff**2))
    outMAE = np.mean(np.abs(outdiff))
    print(f"out of sample\n   RMSE:\n      {outRMSE}")
    print(f"   MAE:\n      {outMAE}")

0  2024-12-03 115353819604 inital cboe spx relative asian
1  2024-12-03 221708375538 inital cboe spx relative barrier


select model:  0


In [2]:
model = joblib.load(pickle)

In [3]:
model['mlp_params']

{'solver': 'sgd',
 'activation': 'relu',
 'max_iter': 500,
 'random_state': 1312,
 'early_stopping': True,
 'hidden_layer_sizes': (20,),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.1,
 'n_iter_no_change': 20,
 'tol': 0.0001,
 'warm_start': False}

In [4]:
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
mlp = MLPRegressor(**model['mlp_params'])
identity = Pipeline([
            ("preprocessor", model['preprocessor']),
            ("regressor", mlp)
        ])
print(identity[1].__dict__)
print()
print(identity.fit(model['train_X'],model['train_y']))
print()
print('out activation: ',identity[1].__dict__['out_activation_'])

{'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'batch_size': 'auto', 'learning_rate': 'adaptive', 'learning_rate_init': 0.1, 'power_t': 0.5, 'max_iter': 500, 'loss': 'squared_error', 'hidden_layer_sizes': (20,), 'shuffle': True, 'random_state': 1312, 'tol': 0.0001, 'verbose': False, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': True, 'early_stopping': True, 'validation_fraction': 0.1, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-08, 'n_iter_no_change': 20, 'max_fun': 15000}

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  ['days_to_maturity',
                                                   'fixing_frequency',
                                                   'past_fixings',
                                                   'risk_free_rate',
                                                

In [5]:
mlp = MLPRegressor(**model['mlp_params'])
relu = Pipeline([
            ("preprocessor", model['preprocessor']),
            ("regressor", mlp)
        ])
print(relu[1].__dict__)
print()
print(relu.fit(model['train_X'],model['train_y']))
print()
relu[1].__dict__['out_activation_'] = 'relu'
print('out activation: ',relu[1].__dict__['out_activation_'])

{'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'batch_size': 'auto', 'learning_rate': 'adaptive', 'learning_rate_init': 0.1, 'power_t': 0.5, 'max_iter': 500, 'loss': 'squared_error', 'hidden_layer_sizes': (20,), 'shuffle': True, 'random_state': 1312, 'tol': 0.0001, 'verbose': False, 'warm_start': False, 'momentum': 0.9, 'nesterovs_momentum': True, 'early_stopping': True, 'validation_fraction': 0.1, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-08, 'n_iter_no_change': 20, 'max_fun': 15000}

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  ['days_to_maturity',
                                                   'fixing_frequency',
                                                   'past_fixings',
                                                   'risk_free_rate',
                                                

In [6]:
print('identity:')
test_performance(model,identity)
print('\nrelu:')
test_performance(model,relu)

identity:
in sample
   RMSE:
      0.005038507274035199
   MAE:
      0.003040082877120672
out of sample
   RMSE:
      0.05114941358661329
   MAE:
      0.03269348023657448

relu:
in sample
   RMSE:
      0.005036100109108606
   MAE:
      0.00299173319064644
out of sample
   RMSE:
      0.0353026686355799
   MAE:
      0.016282666365836203
